In [1]:
import requests

In [2]:
# curl
# 'https://osinfo.prefeitura.rio/report/groupedExpenses/server/groupedExpensesService/generateGroupExpensesReportByExpenses'
# -H 'Accept: application/json, text/javascript, */*; q=0.01'
# -H 'Referer: https://osinfo.prefeitura.rio/pages/application-container.html'
# -H 'Origin: https://osinfo.prefeitura.rio'
# -H 'X-Requested-With: XMLHttpRequest'
# -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
# -H 'Content-Type: application/json'
# --data-binary '
# {"cod_os":263,
#  "id_contrato":11,
#  "contas":[],
#  "tipo_unidade":null,
#  "unidades":[],
#  "tipo_relatorio":"EXPENSES",
#  "ref_inicio_mes":"01",
#  "ref_inicio_ano":"2012",
#  "ref_fim_mes":"09",
#  "ref_fim_ano":"2015"}'
# --compressed

In [3]:
url = 'https://osinfo.prefeitura.rio/report/groupedExpenses/server/groupedExpensesService/generateGroupExpensesReportByExpenses'

In [4]:
data = '{"cod_os":263,"id_contrato":11,"contas":[],"tipo_unidade":null,"unidades":[],"tipo_relatorio":"EXPENSES","ref_inicio_mes":"01","ref_inicio_ano":"2012","ref_fim_mes":"09","ref_fim_ano":"2015"}'

In [5]:
headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Referer': 'https://osinfo.prefeitura.rio/pages/application-container.html',
    'Origin': 'https://osinfo.prefeitura.rio',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36',
    'Content-Type': 'application/json',
}

In [6]:
response = requests.post(url, headers=headers, data=data)

In [9]:
response.json()

[{'id_despesa': 1,
  'cod_despesa': '01',
  'despesa': 'PESSOAL',
  'ref_mes': 1,
  'ref_ano': 2012,
  'valor': 4407494.8,
  'flg_ativo': 'S',
  'message': '',
  'tamanhoCabecalho': 45,
  'nivel': '1'},
 {'id_despesa': 1,
  'cod_despesa': '01',
  'despesa': 'PESSOAL',
  'ref_mes': 2,
  'ref_ano': 2012,
  'valor': 4628754.56,
  'flg_ativo': 'S',
  'message': '',
  'tamanhoCabecalho': 45,
  'nivel': '1'},
 {'id_despesa': 1,
  'cod_despesa': '01',
  'despesa': 'PESSOAL',
  'ref_mes': 3,
  'ref_ano': 2012,
  'valor': 4877966.77,
  'flg_ativo': 'S',
  'message': '',
  'tamanhoCabecalho': 45,
  'nivel': '1'},
 {'id_despesa': 1,
  'cod_despesa': '01',
  'despesa': 'PESSOAL',
  'ref_mes': 4,
  'ref_ano': 2012,
  'valor': 4957096.19,
  'flg_ativo': 'S',
  'message': '',
  'tamanhoCabecalho': 45,
  'nivel': '1'},
 {'id_despesa': 1,
  'cod_despesa': '01',
  'despesa': 'PESSOAL',
  'ref_mes': 5,
  'ref_ano': 2012,
  'valor': 4895415.91,
  'flg_ativo': 'S',
  'message': '',
  'tamanhoCabecalho': 45